In [109]:
import torch
import torch.nn as nn

# Importamos dataset y dataloader, para procesar los datos para ingresarlos a los modelos
# Nos permite usar nuestros datos para generar nuestro mdoelo de entrenamiento, agrupa los ejemplos con sus respectivas etiquetas.
# Dataloader, convierte dataset a un objeto iterable para pytorch, nos permite realizar nuestros propios minibatches de forma automatica 
# De forma simple con dataloader
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import numpy as np
import nltk
nltk.download('punkt')
import random
import json
import string
import time


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\feden\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### DEFINE NEURAL NETWORK

In [110]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, hidden_size) 
        self.l4 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.l4(out)
        # no activation and no softmax at the end
        return out
    


### Methods
    For clean data
    Tokenize sentence
    Bag of Words

In [111]:

#REMOVE ACCENTS
def removeAccents(word):
    translation = str.maketrans("áéíóúüàèìùò", "aeiouuaeiou")
    return word.translate(translation)


#REMOVE PUNCTUATION AND SINGS
def removePunctuation (word):
    word = word.translate(str.maketrans('', '', string.punctuation))   
    return word


#TOKENIZE WORDS
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

# BAG OF WORDS
def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """

    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in tokenized_sentence: 
            bag[idx] = 1

    return bag



### LOAD DATA

In [112]:

with open('chitchat_intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
        tag = intent['tag']
        
        # add to tag list
        tags.append(tag)
        for pattern in intent['patterns']:
            
            #Clean data
            # tokenize each word in the sentence
            words = tokenize(pattern)
            
            # Remove accents
            w = [removeAccents(word) for word in words]

            # Remove punctuation
            w = [removePunctuation(word) for word in words]

            # Remove empty slots    
            w = [word for word in words if word != '']
            
            # add to our words list
            all_words.extend(words)
            
            # add to xy pair
            xy.append((words, tag))

# Remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))


print(f"{len(xy)} patterns: {xy}\n")
# print(len(tags), "tags:", tags)
# print("")
# print(len(all_words), " words:", all_words)


56 patterns: [(['hola'], 'CC_HOLA'), (['hi'], 'CC_HOLA'), (['hello'], 'CC_HOLA'), (['ciao'], 'CC_HOLA'), (['buen', 'dia'], 'CC_HOLA'), (['buenas', 'tardes'], 'CC_HOLA'), (['buenas', 'noches'], 'CC_HOLA'), (['hey'], 'CC_HOLA'), (['whats', 'up'], 'CC_HOLA'), (['que', 'onda'], 'CC_HOLA'), (['hola', 'que', 'tal'], 'CC_HOLA'), (['hola', 'amigo'], 'CC_HOLA'), (['hola', 'holaa'], 'CC_HOLA'), (['cya'], 'CC_ADIOS'), (['nos', 'vemos'], 'CC_ADIOS'), (['Goodbye'], 'CC_ADIOS'), (['chau'], 'CC_ADIOS'), (['adios'], 'CC_ADIOS'), (['hasta', 'luego'], 'CC_ADIOS'), (['me', 'voy'], 'CC_ADIOS'), (['Cuantos', 'anios'], 'CC_EDAD'), (['Cuantos', 'aÃ±os', 'tenes'], 'CC_EDAD'), (['cual', 'es', 'tu', 'edad'], 'CC_EDAD'), (['que', 'tan', 'viejo', 'eres'], 'CC_EDAD'), (['edad', '?'], 'CC_EDAD'), (['tu', 'edad'], 'CC_EDAD'), (['dime', 'tus', 'aÃ±os'], 'CC_EDAD'), (['como', 'te', 'apodaron'], 'CC_NAME_BOT'), (['como', 'te', 'llamas'], 'CC_NAME_BOT'), (['cual', 'es', 'tu', 'nombre'], 'CC_NAME_BOT'), (['como', 'te', '

### Prepare Data for model
    80% of the dataset will be for train the model
    20% of the dataset will be for testing the model
    We can use random_split from Pytorch or Split the data manually

In [113]:
# number_rows = len(xy)    # The size of our dataset or the number of rows in excel table.  
# test_split = int(number_rows*0.2)  
# train_split = number_rows - test_split     
# train_set, test_set = random_split(xy, [train_split, test_split])   


print("\n---------------CREATING TESTING DATA--------------------\n")
# Create testing data
# Data for training
x_test = []
y_test = []

# Data for testing the model
test_array = []
percentage = int(len(xy)*0.2)

for i in range(percentage):
#     item = dataset_copy.pop(random.randrange(len(xy)))
    item = xy.pop(random.randrange(len(xy)))
    test_array.append(item)
    bag = bag_of_words(item[0], all_words)
    x_test.append(bag)
    label = tags.index(item[1])
    y_test.append(label)


x_test = np.array(x_test)
y_test = np.array(y_test)

print("\n--------------- TESTING DATA CREATED--------------------\n")



print("\n---------------CREATING TRAINING DATA--------------------\n")
# Create training data
# Data for training
x_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    x_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)


x_train = np.array(x_train)
y_train = np.array(y_train)

print("\n--------------- TRAIN DATA CREATED--------------------\n")



---------------CREATING TESTING DATA--------------------


--------------- TESTING DATA CREATED--------------------


---------------CREATING TRAINING DATA--------------------


--------------- TRAIN DATA CREATED--------------------



### DEFINE CLASSES FOR DATASET
    Train Dataset
    Test Dataset

In [114]:
# Train Class Dataset
class TrainDataset(Dataset):

    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples
    
# Test Class Dataset
class TestDataset(Dataset):

    def __init__(self):
        self.n_samples = len(x_test)
        self.x_data = x_test
        self.y_data = y_test

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

### TRAINING THE MODEL
    Define Hyper Parameters
        Epochs,
        Batch_size,
        Learning_rate,
        input, hidden, output sizes

In [123]:
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(x_train[0])
hidden_size = 8
output_size = len(tags)

# Define Train Loader
train_set = TrainDataset()
train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True, num_workers=0)

# Define Test Loader
test_set = TestDataset()
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True, num_workers=0)
model.eval()
test_loss, correct = 0, 0
size = len(test_loader.dataset)

# Select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create neural network
model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
print("\n--------------- TRAINING DATA --------------------\n")
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
            words = words.to(device)
            labels = labels.to(dtype=torch.long).to(device)

            # Forward pass
            outputs = model(words)

            # if y would be one-hot, we must apply
            # labels = torch.max(labels, 1)[1]
            loss = loss_fn(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    if (epoch+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    

print(f'\nFinal loss: {loss.item():.4f}\n')

trained_data = {
        "model_state": model.state_dict(),
        "input_size": input_size,
        "hidden_size": hidden_size,
        "output_size": output_size,
        "all_words": all_words,
        "tags": tags
}

FILE = "data.pth"
torch.save(trained_data, FILE)

print(f'training complete. file saved to {FILE}')


--------------- TRAINING DATA --------------------

Epoch [100/1000], Loss: 0.3762
Epoch [200/1000], Loss: 0.0243
Epoch [300/1000], Loss: 0.0060
Epoch [400/1000], Loss: 0.0016
Epoch [500/1000], Loss: 0.0010
Epoch [600/1000], Loss: 0.0002
Epoch [700/1000], Loss: 0.0003
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0001
Epoch [1000/1000], Loss: 0.0000

Final loss: 0.0000

training complete. file saved to data.pth


### TEST MODEL
    With pytroch lib

In [126]:
# Test the model
model.eval()
test_loss, correct = 0, 0
size = len(test_loader.dataset)

print("\n--------------- TESTING DATA --------------------\n")
for epoch in range(num_epochs):
    with torch.no_grad():
        for (pattern, label) in test_loader:
                pattern = pattern.to(device)
                label = label.to(dtype=torch.long).to(device)

                pred = model(pattern)

                test_loss += loss_fn(pred, label).item()
                correct += (pred.argmax(1) == label).type(torch.float).sum().item()
                
        test_loss /= batch_size
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
     


--------------- TESTING DATA --------------------

Test Error: 
 Accuracy: 54.5%, Avg loss: 0.679010 

Test Error: 
 Accuracy: 59.5%, Avg loss: 0.898093 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.580189 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.684455 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.697466 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.765797 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.816899 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.612771 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.544695 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.543839 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.924733 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.897536 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.891080 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.937306 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.619568 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.549113 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.540895 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.888064 

Test Error: 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.859151 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.923666 

Test Error: 
 Accuracy: 60.0%, Avg loss: 1.112003 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.606928 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.854754 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.718776 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.701585 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.703198 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.702887 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.737583 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.948951 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.970216 

Test Error: 
 Accuracy: 60.0%, Avg loss: 1.047054 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.813141 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.924090 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.587178 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.890688 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.825274 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.749804 

Test Error: 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.587003 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.583863 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.575923 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.757840 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.710135 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.700109 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.773363 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.810180 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.600032 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.864206 

Test Error: 
 Accuracy: 60.0%, Avg loss: 1.037470 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.741593 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.878846 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.616869 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.755723 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.593225 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.856097 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.816854 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.747755 

Test Error: 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.957894 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.735142 

Test Error: 
 Accuracy: 60.0%, Avg loss: 1.020748 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.774260 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.909999 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.899190 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.901601 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.584807 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.541006 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.781135 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.709574 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.871230 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.615382 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.544828 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.566841 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.893280 

Test Error: 
 Accuracy: 60.0%, Avg loss: 1.075840 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.602556 

Test Error: 
 Accuracy: 60.0%, Avg loss: 0.687251 

Test Error: 

### TEST MODEL
    Testing Manually

In [127]:
total = 0
correcta = 0
with torch.no_grad():
    for item in test_array:

        #Clean data
        # tokenize each word in the sentence
        pattern = item[0]
        
        # Remove accents
        pattern = [removeAccents(word) for word in pattern]

        # Remove punctuation
        pattern = [removePunctuation(word) for word in pattern]

        # Remove empty slots    
        pattern = [word for word in pattern if word != '']
            

        X = bag_of_words(pattern, all_words)
        X = X.reshape(1, X.shape[0])
        X = torch.from_numpy(X).to(device)

        output = model(X)
        _, predicted = torch.max(output, dim=1)
        
        tag = tags[predicted.item()]

        probs = torch.softmax(output, dim=1)
        prob = probs[0][predicted.item()]
        
        print(f"EJEMPLO: {item[0]} - Intencion:{item[1]}")
        print(f"PREDICCION:\n Intencion:{tag}\n Confianza:{prob.item()} \n")
        total += prob.item()
        if(tag == item[1]):
            correcta += 1
            
print("Accuracy de tu modelo:",((total / len(test_array) * int(correcta/0.1)) / 100))
print(f"{correcta}/{len(test_array)} acertadas")
print(f"Clasifico correctamente el {int(correcta/0.1)}% de los ejemplos para testing")

EJEMPLO: ['a', 'que', 'te', 'dedicas'] - Intencion:CC_PARA_QUE_FUE_CREADO
PREDICCION:
 Intencion:CC_HOLA
 Confianza:0.9993878602981567 

EJEMPLO: ['que', 'onda'] - Intencion:CC_HOLA
PREDICCION:
 Intencion:CC_PARA_QUE_FUE_CREADO
 Confianza:0.9986312985420227 

EJEMPLO: ['hola', 'amigo'] - Intencion:CC_HOLA
PREDICCION:
 Intencion:CC_HOLA
 Confianza:0.9999898672103882 

EJEMPLO: ['sos', 'feliz'] - Intencion:CC_COMO_ESTAS
PREDICCION:
 Intencion:CC_NAME_BOT
 Confianza:0.6260813474655151 

EJEMPLO: ['cual', 'es', 'tu', 'funcion'] - Intencion:CC_PARA_QUE_FUE_CREADO
PREDICCION:
 Intencion:CC_HOLA
 Confianza:0.989615797996521 

EJEMPLO: ['hey'] - Intencion:CC_HOLA
PREDICCION:
 Intencion:CC_ADIOS
 Confianza:0.6899569630622864 

EJEMPLO: ['cual', 'es', 'tu', 'nombre'] - Intencion:CC_NAME_BOT
PREDICCION:
 Intencion:CC_NAME_BOT
 Confianza:0.8743694424629211 

EJEMPLO: ['estas', 'contento'] - Intencion:CC_COMO_ESTAS
PREDICCION:
 Intencion:CC_COMO_ESTAS
 Confianza:0.8686397075653076 

EJEMPLO: ['esta

### CHAT

In [128]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('chitchat_intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "WAMAC"
print("Hola! hablemos un rato... (type 'quit' to exit)")
while True:
    sentence = input("Tu: ")

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    print(f"Confianza:{prob.item()} -> Intencion:{tag}")

    if prob.item() > 0.7:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
        if tag == "CC_ADIOS":
            break
    else:
        print(f"{bot_name}: No te entendi...")

Hola! hablemos un rato... (type 'quit' to exit)
Tu: hola
Confianza:0.9999779462814331 -> Intencion:CC_HOLA
WAMAC: Hola, me alegra verte de nuevo por aqui
Tu: que haces
Confianza:0.999941349029541 -> Intencion:CC_PARA_QUE_FUE_CREADO
WAMAC: Fui creado para resolver dudas.
Tu: como te llamas
Confianza:0.9999778270721436 -> Intencion:CC_NAME_BOT
WAMAC: Seguro tu me puedes ayudar con eso... era WAMAC, verdad?
Tu: tu nombre
Confianza:0.9980487823486328 -> Intencion:CC_NAME_BOT
WAMAC: No se si ya me pusieron mi nombre, pero escuche rumores de que me llamo WAMAC.
Tu: nombre
Confianza:0.9943636655807495 -> Intencion:CC_NAME_BOT
WAMAC: No se si ya me pusieron mi nombre, pero escuche rumores de que me llamo WAMAC.
Tu: que frio hace hoy
Confianza:0.9999954700469971 -> Intencion:CC_EL_CLIMA
WAMAC: Pues no se, no he aprendido sobre el clima todavia
Tu: hola quiero saber la temperatura
Confianza:0.8324239253997803 -> Intencion:CC_ADIOS
WAMAC: Adios!
